In [1]:
import pysb.bng
import kholodenko_basic as kb
import kholodenko_m1 as ke
import sympy as sp

pysb.bng.__file__

'/home/priya/pysb/pysb/bng.pyc'

In [2]:
pysb.bng.generate_equations(kb.model)
pysb.bng.generate_equations(ke.model)

In [3]:
for i, j in zip(kb.model.species, ke.model.species):
    print i, j
    assert i.is_equivalent_to(j)

species_map = dict(zip(range(len(kb.model.species)), kb.model.species))
def species(tup):
    return [species_map[x] for x in tup]
#print ke.model.parameters
#print kb.model.parameters
#print kb.model.expressions
ke_subs, kb_subs = [zip(m.model.expressions,[e.expand_expr(expand_observables=True) for e in m.model.expressions]) for m in ke, kb]
print ke.model.parameters[2]
ke_dict, kb_dict = [dict([((x['reactants'], x['products']), x['rate']) for x in y])
                                    for y in [ke.model.reactions, kb.model.reactions]]
for key in sorted(ke_dict.keys()):
    e_sub = sp.sympify(str(ke_dict[key].subs(ke_subs).simplify()))
    b_sub = sp.sympify(str(kb_dict[key].subs(kb_subs).simplify()))
    print ""
    print "rct:", key[0],"-->", key[1]
    print "rct:", species(key[0]),"-->", species(key[1])
    print "e", e_sub
    print "b", b_sub
    print ""
    #assert e_sub.equals(b_sub)

    
    #print e.subs(ke_subs), b.subs(kb_subs)
#for e, b in rct:
    #e_sub = sp.sympify(str(e[2].subs(ke_subs).simplify()))
    #b_sub = sp.sympify(str(b[2].subs(kb_subs).simplify()))

B(d=None, i=None) B(d=None, i=None)
I(b=None) I(b=None)
B(d=1, i=None) % B(d=1, i=None) B(d=1, i=None) % B(d=1, i=None)
B(d=None, i=1) % I(b=1) B(d=None, i=1) % I(b=1)
B(d=1, i=2) % B(d=1, i=None) % I(b=2) B(d=1, i=2) % B(d=1, i=None) % I(b=2)
B(d=1, i=2) % B(d=1, i=3) % I(b=2) % I(b=3) B(d=1, i=2) % B(d=1, i=3) % I(b=2) % I(b=3)
Parameter('RT', 2.577)

rct: (0, 0) --> (2,)
rct: [B(d=None, i=None), B(d=None, i=None)] --> [B(d=1, i=None) % B(d=1, i=None)]
e 0.5*__s0**2*kon_BB
b 0.5*__s0**2*kon_BB


rct: (0, 1) --> (3,)
rct: [B(d=None, i=None), I(b=None)] --> [B(d=None, i=1) % I(b=1)]
e __s0*__s1*kon_BI
b __s0*__s1*kon_BI


rct: (0, 3) --> (4,)
rct: [B(d=None, i=None), B(d=None, i=1) % I(b=1)] --> [B(d=1, i=2) % B(d=1, i=None) % I(b=2)]
e __s0*__s3*kon_BB*exp(-phi*log(f))
b __s0*__s3*f**(-phi)*kon_BB


rct: (1, 2) --> (4,)
rct: [I(b=None), B(d=1, i=None) % B(d=1, i=None)] --> [B(d=1, i=2) % B(d=1, i=None) % I(b=2)]
e 2.0*__s1*__s2*kon_BI*exp(-phi*log(f))
b 2.0*__s1*__s2*f**(-phi)*kon_BI


### dir(m)

In [4]:
print ke.model.reactions


[{'reactants': (0, 0), 'products': (2,), 'rate': 0.5*__s0**2*_BB_dimerization_local1, 'rule': ('BB_dimerization',), 'reverse': (False,)}, {'reactants': (0, 1), 'products': (3,), 'rate': __s0*__s1*_BI_binding_local1, 'rule': ('BI_binding',), 'reverse': (False,)}, {'reactants': (0, 3), 'products': (4,), 'rate': __s0*__s3*_BB_dimerization_local2, 'rule': ('BB_dimerization',), 'reverse': (False,)}, {'reactants': (3, 3), 'products': (5,), 'rate': 0.5*__s3**2*_BB_dimerization_local3, 'rule': ('BB_dimerization',), 'reverse': (False,)}, {'reactants': (2,), 'products': (0, 0), 'rate': __s2*__reverse_BB_dimerization_local1, 'rule': ('BB_dimerization',), 'reverse': (True,)}, {'reactants': (1, 2), 'products': (4,), 'rate': 2.0*__s1*__s2*_BI_binding_local2, 'rule': ('BI_binding',), 'reverse': (False,)}, {'reactants': (3,), 'products': (0, 1), 'rate': __s3*__reverse_BI_binding_local1, 'rule': ('BI_binding',), 'reverse': (True,)}, {'reactants': (4,), 'products': (0, 3), 'rate': __s4*__reverse_BB_dime

In [5]:
sp.parsing.sympy_parser.eval_expr('0.5*__s0**2*exp(RT*log(kon_BB)*RT**(-1))', {}, {})

NameError: name '__s0' is not defined